In [54]:
# read html file and get table
import bs4
import re
import pandas as pd
import numpy as np
note_map = {
    "0": 0,
    "37": 1,
    "51": 2,
    "65": 3,
    "79": 4,
    "93": 5,
    "107": 6,
    "121": 7
}
file_path = 'test2.html'
fumen = pd.DataFrame()
read_file = open(file_path, 'r')
soup = bs4.BeautifulSoup(read_file, 'html.parser')

In [65]:
def parse(content):
    # 0     1    2   3   4   5   6   7   8   9
    # 皿    1    2   3   4   5   6   7   BPM 小节
    if content.name == 'img':
        if content.get('src') == '../t.gif':    # BPM小节线
            reg = re.compile(r'top:\d+')
            top = reg.findall(content.get('style'))[0]
            pass
        elif content.get('src') == '../b.gif' or content.get('src') == '../w.gif': # 黑白键
            # print(content.get('style'))
            # use regex to get the two numbers
            reg = re.compile(r'\d+')
            top, left = reg.findall(content.get('style'))
            left = note_map[left]
            return(int(top) + 4, 1, int(left)) # +4 因为图片高度为4
        elif content.get('src') == '../s.gif': # 皿
            reg = re.compile(r'\d+')
            top, left = reg.findall(content.get('style'))
            left = note_map[left]
            return(int(top) + 4, 1, left) # +4 因为图片高度为4
    elif content.name == 'span':    # BPM数字，top+5为小节线位置，理论上图片高度为2，但+1即offset到0-127，使变速线在中间
        print(content.text)
        reg = re.compile(r'top:\d+')
        top = int(reg.findall(content.get('style'))[0][4:])
        return(top+5+1, int(content.text), 8)
    
class fumen():
    def __init__(self):
        self.data = pd.DataFrame()
    def add(self, mes):
        self.data = self.data.append(mes)


In [66]:
# get table in soup 
soup_table = soup.find_all('table')

In [68]:
spfumen = fumen()

for table in soup_table[1:]: # 第一张表是原表
    len(table.find_all('tr'))
    for tr in table.find_all('tr'):
        df = pd.DataFrame(np.zeros((128, 10)), dtype=int)
        for notes in tr.find('div').contents:
            print(notes)
            
            location = parse(notes)
            print(parse(notes))

            if location is None:
                continue
            else:
                y, n, x = location
                df.iloc[y, x] = n
        mes = tr.find('th').text
        df.iloc[:, 9] = mes
        spfumen.add(df)        


<img name="ttl1" onmousedown="dragOn('ttl1')" src="../w.gif" style="top:59px;left:65px"/>
(63, 1, 3)
<img name="ttl2" onmousedown="dragOn('ttl2')" src="../b.gif" style="top:43px;left:107px"/>
(47, 1, 6)
<img name="ttl3" onmousedown="dragOn('ttl3')" src="../b.gif" style="top:27px;left:51px"/>
(31, 1, 2)
<img name="ttl4" onmousedown="dragOn('ttl4')" src="../w.gif" style="top:27px;left:93px"/>
(31, 1, 5)
<img name="ttl5" onmousedown="dragOn('ttl5')" src="../s.gif" style="top:11px;left:0px"/>
(15, 1, 0)
<img height="2" src="../t.gif" style="top:126px" width="134"/>
None
<span style="z-index:5;font-size:10px;top:121px;font-weight:bold;width:20px;text-align:center;background:gray;left:140px;color:palegreen;">290</span>
290
290
(127, 290, 8)
<img name="ttl6" onmousedown="dragOn('ttl6')" src="../w.gif" style="top:123px;left:65px"/>
(127, 1, 3)
<img name="ttl7" onmousedown="dragOn('ttl7')" src="../w.gif" style="top:115px;left:37px"/>
(119, 1, 1)
<img name="ttl8" onmousedown="dragOn('ttl8')" src

In [73]:
spfumen.data[1:30]

,0,1,2,3,4,5,6,7,8,9
1,0,0,0,0,0,0,0,0,0,5
2,0,0,0,0,0,0,0,0,0,5
3,0,0,0,0,0,0,0,0,0,5
4,0,0,0,0,0,0,0,0,0,5
5,0,0,0,0,0,0,0,0,0,5
6,0,0,0,0,0,0,0,0,0,5
7,0,0,0,0,0,0,0,0,0,5
8,0,0,0,0,0,0,0,0,0,5
9,0,0,0,0,0,0,0,0,0,5
10,0,0,0,0,0,0,0,0,0,5


In [23]:
'top:123px;left:79px'
# reg: get top and left
# top: (\d+)px;left: (\d+)px
re.findall(r'top: (\d+)px;left: (\d+)px', s[1].get("style"))

NameError: name 's' is not defined

# 画图

In [118]:
import conventor
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import math

In [135]:
fumen = conventor.fumen()
fumen.read('test2.html')
data = fumen.data

<img name="ttl1" onmousedown="dragOn('ttl1')" src="../w.gif" style="top:59px;left:65px"/>
(63, 1, 3)
<img name="ttl2" onmousedown="dragOn('ttl2')" src="../b.gif" style="top:43px;left:107px"/>
(47, 1, 6)
<img name="ttl3" onmousedown="dragOn('ttl3')" src="../b.gif" style="top:27px;left:51px"/>
(31, 1, 2)
<img name="ttl4" onmousedown="dragOn('ttl4')" src="../w.gif" style="top:27px;left:93px"/>
(31, 1, 5)
<img name="ttl5" onmousedown="dragOn('ttl5')" src="../s.gif" style="top:11px;left:0px"/>
(15, 1, 0)
<img height="2" src="../t.gif" style="top:126px" width="134"/>
None
<span style="z-index:5;font-size:10px;top:121px;font-weight:bold;width:20px;text-align:center;background:gray;left:140px;color:palegreen;">290</span>
290
290
(127, 290, 8)
<img name="ttl6" onmousedown="dragOn('ttl6')" src="../w.gif" style="top:123px;left:65px"/>
(127, 1, 3)
<img name="ttl7" onmousedown="dragOn('ttl7')" src="../w.gif" style="top:115px;left:37px"/>
(119, 1, 1)
<img name="ttl8" onmousedown="dragOn('ttl8')" src

In [136]:
# decide the size of image
## find max in data column 9
max_mes = data.iloc[:, 9].max()
width = math.ceil(max_mes/4) * (134+32 + 25)
height = 128 * 4 + 100


In [138]:
# create image
def drawer(rail, mes, note, top):
    # load note image
    note_img_map = {
        0: 'pic/s.gif',
        1: 'pic/w.gif',
        2: 'pic/b.gif',        
        3: 'pic/w.gif',
        4: 'pic/b.gif',        
        5: 'pic/w.gif',
        6: 'pic/b.gif',        
        7: 'pic/w.gif',
    }
    rail_x = {
        0: 0,
        1: 36,
        2: 49,
        3: 62,
        4: 75,
        5: 88,
        6: 101,
        7: 114
    }
    note_img = Image.open(note_img_map[rail])
    note_img = note_img.resize([note_img.size[0]*4, note_img.size[1]*4])

    # from mes and top, get x and y
    column = mes//4
    row = 3-mes%4
    x = column * (backsize_x + greysize_x) + rail_x[rail]*4
    y = row * (greysize_y) + top*4
    return note_img, (x, y)

## variables
single_backsize_x = 134
single_backsize_y = 32

backsize_x = single_backsize_x*4
backsize_y = single_backsize_y*4
greysize_x = 32*4
greysize_y = backsize_y*4
messize_x = backsize_x + greysize_x
messize_y = greysize_y
grey = (128, 128, 128)
white = (255, 255, 255)
black = (0, 0, 0)

img = Image.new('RGB', (width*4, height*4), black)

back = Image.open('pic/70.gif') # size: 134*32
back = back.resize((134*4, 32*4))
back = back.transpose(Image.FLIP_LEFT_RIGHT)
background = Image.new('RGB', (messize_x, messize_y), black)
background.paste(back, (0, 0))
background.paste(back, (0, backsize_y))
background.paste(back, (0, backsize_y*2))
background.paste(back, (0, backsize_y*3))
draw_back = ImageDraw.Draw(background)
draw_back.rectangle((backsize_x, 0, backsize_x + greysize_x, greysize_y), outline=grey, fill=grey)
draw_back.rectangle((0, 0, backsize_x + greysize_x, greysize_y), outline=white, width=3)


draw = ImageDraw.Draw(img)

for mes in range(max_mes):
    # get x and y of mes
    column = mes//4
    row = 3-mes%4
    x = column * (backsize_x + greysize_x)
    y = row * (greysize_y)
    print(x, y)
    img.paste(background, (x, y))

for index, row in data.iterrows():
    for key_i in range(8):
        if row[key_i] != 0:
            print("rail =" + str(key_i) + "mes =" + str(row[9]) + "note =" + str(row[key_i]) + "top =" + str(index))
            
            note_img, location = drawer(key_i, row[9], row[key_i], index)
            img.paste(note_img, location)
# draw background
img.show()


C:\Users\Komeiji Satori\AppData\Local\Temp\ipykernel_24632\209618700.py:52: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  back = back.transpose(Image.FLIP_LEFT_RIGHT)


0 1536
0 1024
0 512
0 0
664 1536
664 1024
664 512
664 0
1328 1536
1328 1024
1328 512
1328 0
1992 1536
1992 1024
1992 512
1992 0
2656 1536
2656 1024
2656 512
2656 0
3320 1536
3320 1024
3320 512
3320 0
3984 1536
3984 1024
3984 512
3984 0
4648 1536
4648 1024
4648 512
4648 0
5312 1536
5312 1024
5312 512
5312 0
5976 1536
5976 1024
5976 512
5976 0
6640 1536
6640 1024
6640 512
6640 0
7304 1536
7304 1024
7304 512
7304 0
7968 1536
7968 1024
7968 512
7968 0
8632 1536
8632 1024
8632 512
8632 0
9296 1536
9296 1024
9296 512
9296 0
9960 1536
9960 1024
9960 512
9960 0
10624 1536
10624 1024
10624 512
10624 0
11288 1536
11288 1024
11288 512
11288 0
11952 1536
11952 1024
11952 512
11952 0
12616 1536
12616 1024
12616 512
12616 0
13280 1536
13280 1024
13280 512
13280 0
13944 1536
13944 1024
13944 512
13944 0
14608 1536
14608 1024
14608 512
14608 0
15272 1536
15272 1024
15272 512
15272 0
15936 1536
15936 1024
15936 512
15936 0
16600 1536
16600 1024
16600 512
16600 0
17264 1536
17264 1024
17264 512
17264 0


In [139]:
img.save('test2.png')

In [91]:
data.iloc[129][1]

0

In [110]:
img.size

(29032, 2448, 29032, 2448, 29032, 2448, 29032, 2448)